In [96]:
from pyVHR.datasets.ubfc2 import UBFC2
from pyVHR.datasets.dataset import Dataset
from pyVHR.signals.video import Video
from copy import copy
import numpy as np

import os
#RUN ON CPU 
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

#Tensorflow/KERAS
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.models import model_from_json
from tensorflow.python.keras.layers import ZeroPadding3D, Dense, Activation,Conv3D,MaxPooling3D,AveragePooling3D,Flatten,Dropout
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.models import model_from_json


In [64]:
# -- dataset object
dataset = UBFC2()

# -- ground-truth (GT) signal
fname = "./UBFC/DATASET_2/subject1/ground_truth.txt"

# -- load signal and build a BVPsignal or ECGsignal object
sigGT = dataset.readSigfile(fname)

# -- compute BPM GT
winSizeGT = 2
bpmGT, timesGT = sigGT.getBPM(winSizeGT)


In [72]:
bpm = np.round(bpmGT)
bpm =bpm - 55
bpm = np.round(bpm / 2.5)
print(bpm)

[-2. 12. 16. 15. 18. 16. 18. 14. 18. 18. 22. 20. 26. 16. 18. 20. 20. 19.
 22. 23. 24. 20. 18. 22. 26. 19. 18. 20. 15. 19. 21. 23. 24. 21. 22. 18.
 22. 17. 23. 17. 24. 23. 25. 20. 15. 20. 18. 21. 23. 22. 20. 22. 19. 21.
 19.]


In [66]:
print(timesGT)

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53.
 54.]


In [105]:
LENGTH_VIDEO = 60 
IMAGE_WIDTH = 25 
IMAGE_HEIGHT = 25 
IMAGE_CHANNELS = 1 
RATE = 30
NB_SECOND = int(LENGTH_VIDEO / RATE)


videoFilename = "./UBFC/DATASET_2/subject1/vid.avi"
video = Video(videoFilename)
video.getCroppedFaces(detector='dlib', extractor='skvideo')
video.setMask(typeROI='skin_adapt',skinThresh_adapt=0.22)

NB_LAPSE = int(video.numFrames / RATE)

imgs = np.zeros(shape=(video.numFrames, video.cropSize[0], video.cropSize[1], 1))
xtest = np.zeros(shape=(NB_LAPSE, LENGTH_VIDEO, IMAGE_HEIGHT , IMAGE_WIDTH, 1))
ytest = np.zeros(shape=(NB_LAPSE, NB_CLASSES + 1))

# Available Outputs
HEART_RATES = np.linspace(55, 240, 75)
NB_CLASSES = len(HEART_RATES)

# prepare labels and label categories
labels = np.zeros(NB_CLASSES + 1)

for i in range(NB_CLASSES + 1):
    labels[i] = i
labels_cat = np_utils.to_categorical(labels)
 
# channel extraction
if (video.cropSize[2]<3):
    IMAGE_CHANNELS = 1
else:
    IMAGE_CHANNELS = video.cropSize[2]

# load images (imgs contains the whole video)
for j in range(video.numFrames):

    if (IMAGE_CHANNELS==3):
        temp = video.faces[j]/255
        temp = temp[:,:,1]      # only the G component is currently used
    else:
        temp = video.faces[j] / 255

    imgs[j] = np.expand_dims(temp, 2)
    


for lapse in range(NB_LAPSE):  
    xtemp = np.zeros(shape=(LENGTH_VIDEO, IMAGE_HEIGHT , IMAGE_WIDTH, 1)) 
    
    start = lapse * RATE
    end = start + LENGTH_VIDEO
    if(end > video.numFrames):
        break
    c=0
    
    for j in range(start,end-1):    
        faceCopy = copy(imgs[j])
        #randomization pixel choice 
        np.random.shuffle(faceCopy)
        for m in range(0, IMAGE_HEIGHT):
            for n in range(0, IMAGE_WIDTH):
                xtemp[c][m][n]= faceCopy[m][n]
        c = c +1
                
    xtest[lapse] = np.expand_dims(xtemp, 0)
    ytest[lapse] = np.expand_dims(labels_cat[int(bpm[lapse+NB_SECOND])],0)
    

np.savez('./trainingData.npz', a=xtest, b=ytest)

In [114]:
model = model_from_json(open('./model/model_conv3D.json').read())
model.load_weights('./model/weights_conv3D.h5')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


#start training
model.fit(xtest, ytest, epochs = 100, batch_size=64, verbose=1)


Epoch 1/100
1/1 [==============================] - 0s 0s/step - loss: 2240.4094 - accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 0s/step - loss: 1609.1538 - accuracy: 0.0784
Epoch 3/100
1/1 [==============================] - 0s 0s/step - loss: 1214.0879 - accuracy: 0.1176
Epoch 4/100
1/1 [==============================] - 0s 1ms/step - loss: 1096.6233 - accuracy: 0.0784
Epoch 5/100
1/1 [==============================] - 0s 3ms/step - loss: 776.6274 - accuracy: 0.0392
Epoch 6/100
1/1 [==============================] - 0s 1ms/step - loss: 626.6080 - accuracy: 0.0980
Epoch 7/100
1/1 [==============================] - 0s 0s/step - loss: 513.8994 - accuracy: 0.0980
Epoch 8/100
1/1 [==============================] - 0s 3ms/step - loss: 516.4780 - accuracy: 0.0980
Epoch 9/100
1/1 [==============================] - 0s 0s/step - loss: 565.5861 - accuracy: 0.1176
Epoch 10/100
1/1 [==============================] - 0s 0s/step - loss: 572.5571 - accuracy: 0.1176
Epoch 1